In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from scipy.signal import find_peaks
#from sklearn.neighbors import KernelDensity

In [2]:
gyro = pd.read_csv("data/android.sensor.gyroscope.csv")
accel = pd.read_csv("data/android.sensor.accelerometer.csv")
beats = [float(line.rstrip()) for line in open("data/beats.txt")]

In [24]:
accel_time = accel.iloc[:,0] / 1000000000
gyro_time = gyro.iloc[:,0] / 1000000000
offset_seconds = 11.541678

In [55]:
accel_peaks_x , temp = find_peaks(accel.iloc[:,1],prominence=(25,None))
accel_peaks_x_prominence = temp["prominences"]
accel_peaks_x = accel_time[accel_peaks_x]

accel_valleys_x , temp = find_peaks(-1*accel.iloc[:,1],prominence=(25,None))
accel_valleys_x_prominence = temp["prominences"]
accel_valleys_x = accel_time[accel_valleys_x]

accel_peaks_y , temp = find_peaks(accel.iloc[:,2],prominence=(25,None))
accel_peaks_y_prominence = temp["prominences"]
accel_peaks_y = accel_time[accel_peaks_y]

accel_valleys_y , temp = find_peaks(-1*accel.iloc[:,2],prominence=(25,None))
accel_valleys_y_prominence = temp["prominences"]
accel_valleys_y = accel_time[accel_valleys_y]

accel_peaks_z , temp = find_peaks(accel.iloc[:,3],prominence=(25,None))
accel_peaks_z_prominence = temp["prominences"]
accel_peaks_z = accel_time[accel_peaks_z]

accel_valleys_z , temp = find_peaks(-1*accel.iloc[:,3],prominence=(25,None))
accel_valleys_z_prominence = temp["prominences"]
accel_valleys_z = accel_time[accel_valleys_z]

gyro_peaks_x , temp = find_peaks(gyro.iloc[:,1],prominence=(None,None))
gyro_peaks_x_prominence = temp["prominences"]
gyro_peaks_x = gyro_time[gyro_peaks_x]

gyro_valleys_x , temp = find_peaks(-1*gyro.iloc[:,1],prominence=(None,None))
gyro_valleys_x_prominence = temp["prominences"]
gyro_valleys_x = gyro_time[gyro_valleys_x]

gyro_peaks_y , temp = find_peaks(gyro.iloc[:,2],prominence=(None,None))
gyro_peaks_y_prominence = temp["prominences"]
gyro_peaks_y = gyro_time[gyro_peaks_y]

gyro_valleys_y , temp = find_peaks(-1*gyro.iloc[:,2],prominence=(None,None))
gyro_valleys_y_prominence = temp["prominences"]
gyro_valleys_y = gyro_time[gyro_valleys_y]

gyro_peaks_z , temp = find_peaks(gyro.iloc[:,3],prominence=(None,None))
gyro_peaks_z_prominence = temp["prominences"]
gyro_peaks_z = gyro_time[gyro_peaks_z]

gyro_valleys_z , temp = find_peaks(-1*gyro.iloc[:,3],prominence=(None,None))
gyro_valleys_z_prominence = temp["prominences"]
gyro_valleys_z = gyro_time[gyro_valleys_z]

In [97]:
def find_most_prominent_peak(peaks,peaks_prominences,valleys,valleys_prominences,min_threshold = 0,max_treshold = inf):
    max_prominence = 0
    max_peak = 0
    offset = (min_threshold + max_treshold) / 2
    for peak, prominence in zip(peaks, peaks_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    for peak, prominence in zip(valleys, valleys_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    return (max_peak, max_prominence)
    
def calc_prominence_factor(x,threshold = 0.2,offset=0):
    return exp(0-100*(x-offset)**2)

In [98]:
min_threshold = 0
max_threshold = inf

beat_diff = (beats[0] + beats[-1])/ (len(beats)-1)
threshold = beat_diff / 2

accel_peaks = []
gyro_peaks = []

for beat, i in zip(beats,range(len(beats))):

    #if i < (len(beats)-1):
    #    max_threshold = (beats[i+1] + beat) / 2
    #else: 
    #    max_threshold = inf

    min_threshold = beat - threshold
    max_threshold = beat + threshold

    accel_peaks_with_prominence = []
    gyro_peaks_with_prominence = []


    
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_x,accel_peaks_x_prominence,accel_valleys_x,accel_valleys_x_prominence,min_threshold, max_threshold))
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_y,accel_peaks_y_prominence,accel_valleys_y,accel_valleys_y_prominence,min_threshold, max_threshold))
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_z,accel_peaks_z_prominence,accel_valleys_z,accel_valleys_z_prominence,min_threshold, max_threshold))

    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_x,gyro_peaks_x_prominence,gyro_valleys_x,gyro_valleys_x_prominence,min_threshold, max_threshold))
    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_y,gyro_peaks_y_prominence,gyro_valleys_y,gyro_valleys_y_prominence,min_threshold, max_threshold))
    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_z,gyro_peaks_z_prominence,gyro_valleys_z,gyro_valleys_z_prominence,min_threshold, max_threshold))

    max_accel_prominence = 0
    for _, prominence in accel_peaks_with_prominence:
        if prominence > max_accel_prominence:
            max_accel_prominence = prominence

    accel_peak_sum = 0
    accel_peak_count = 0
    
    for peak, prominence in accel_peaks_with_prominence:
        if prominence  >= (max_accel_prominence * 0.75):
            accel_peak_sum += peak
            accel_peak_count += 1
    if accel_peak_sum > 0:
        accel_peak = accel_peak_sum / accel_peak_count
        accel_peaks.append((accel_peak,max_accel_prominence,accel_peak_count,beat))

    min_threshold = max_threshold


In [102]:
plt.close()
plt.axhline(beat_diff)
plt.axhline(threshold)
#plt.axhline(-1*beat_diff)
#plt.axhline(-1*threshold)
for peak, prominence , _ ,beat in accel_peaks:
    diff = abs(peak - beat)
    plt.plot(beat, diff,"X")
plt.grid(True)
plt.show()